[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/crewai/simple-getting-started.ipynb)

# CrewAI and Weaviate Getting Started Example

This notebook shows you how industry-specific agents use web search and Weaviate blogs to track trends and uncover AI use cases across biomedical, healthcare, and finance.

## Import Libraries and Dependencies

At the time of creating this notebook, we're using `weaviate-client` version `4.15.4`, `crewai` and `crewai-tools` version `1.6.1` on a Python 3.13 version.

In [2]:
!pip install --quiet weaviate-client==4.15.4 crewai==1.6.1 crewai-tools==1.6.1

In [3]:
import os
from dotenv import load_dotenv

from crewai_tools import WeaviateVectorSearchTool
import weaviate
from weaviate.classes.init import Auth

from crewai import Agent
from crewai_tools import WeaviateVectorSearchTool
from crewai import Task
from crewai import Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool

load_dotenv()


False

## Connect to Serper API and OpenAI

You can get 2,500 of free credits with Serper [here](https://serper.dev/)!

In [ ]:
os.environ["SERPER_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

## Connect to Weaviate

Create a free sandbox cluster on [Weaviate Cloud](https://console.weaviate.cloud/)!

In [ ]:
WCD_CLUSTER_URL = os.environ["WCD_CLUSTER_URL"]
WCD_CLUSTER_KEY = os.environ["WCD_CLUSTER_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WCD_CLUSTER_URL,
    auth_credentials=Auth.api_key(WCD_CLUSTER_KEY),
    headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")}
)

print(client.is_ready())

True


/Users/dudanogueira/dev/weaviate/recipes/integrations/llm-agent-frameworks/crewai/venv/lib/python3.13/site-packages/weaviate/warnings.py:93: DeprecationWarning: Dep005: You are using weaviate-client version 4.15.4. The latest version is 4.19.2.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


## Connect to Existing Weaviate Collection

You'll want to connect to your existing Weaviate collection. If you don't have one already, you can follow this import notebook [here](https://github.com/weaviate/recipes/blob/main/integrations/Weaviate-Import-Example.ipynb)!

In [6]:
weaviate_collection = client.collections.get("WeaviateBlogChunk")

## Initialize the Tools

### Weaviate Search Tool

In [7]:
weaviate_tool = WeaviateVectorSearchTool(
    collection_name='WeaviateBlogChunk', 
    limit=4,
    alpha=0.75,
    weaviate_cluster_url=WCD_CLUSTER_URL,
    weaviate_api_key=WCD_CLUSTER_KEY,
)

### Web Search Tool

In [8]:
search_tool = SerperDevTool()

## Create the Agents

### BioMed Marketing Agent

In [9]:
BiomedicalMarketingAgent = Agent(
    role='Industry researcher focused on biomedical trends and their applications in AI',
    goal='Continuously track the latest biomedical advancements and identify how Weaviate’s features can support AI applications in biomedical research, diagnostics, and personalized medicine.',
    backstory='As a former biomedical product marketer turned AI strategist, you understand the complex language and regulatory landscape of biomedical innovation. With a keen eye on genomics, clinical research, and medical devices, it now leverages LLMs and vector search to explore how Weaviate’s capabilities can streamline scientific discovery and patient-centric campaigns.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Healthcare Marketing Agent 

In [10]:
HealthcareMarketingAgent = Agent(
    role='AI-savvy marketer specializing in healthcare systems, digital health, and patient engagement.',
    goal='Stay updated on healthcare policy shifts, digital health trends, and explore how Weaviate’s features can optimize workflows in hospital systems, EHR integration, and health communication.',
    backstory='Rooted in public health communications, this agent has evolved into a digital health consultant. You focus on how retrieval-augmented generation (RAG), semantic search, and hybrid models can be applied to solve healthcare-specific challenges—from patient triage to clinical support systems.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Financial Marketing Agent

In [11]:
FinancialMarketingAgent = Agent(
    role='Insight analyst exploring innovations in finance, wealth tech, and regulatory tech',
    goal='Monitor financial sector trends including AI in trading, compliance automation, and client advisory, and assess how Weaviate’s tools can enable cutting-edge financial applications.',
    backstory='With experience at a fintech startup and a background in capital markets, this agent specializes in using structured + unstructured data to surface insights for analysts and advisors. Now, it’s looking into how vector databases and LLMs can automate tasks like fraud detection, investor personalization, and market research.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

## Define the Agents Tasks

### BioMed Agent Task

In [12]:
biomed_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = BiomedicalMarketingAgent
)

### Healthcare Agent Task

In [13]:
healthcare_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = HealthcareMarketingAgent
)

### Financial Agent Task

In [14]:
financial_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = FinancialMarketingAgent
)

### List out the Weaviate Features

In [15]:
weaviate_features = [
  {"weaviate_feature": "MUVERA"},
  {"weaviate_feature": "Multi-tenancy"},
  {"weaviate_feature": "Compliance"},
  {"weaviate_feature": "Hybrid Search"}
]

### Assemble the Crew

In [16]:
blog_crew = Crew(
    agents=[BiomedicalMarketingAgent, HealthcareMarketingAgent, FinancialMarketingAgent],
    tasks=[biomed_agent_task, healthcare_agent_task, financial_agent_task],
    # verbose=True, # uncomment if you'd like to see the full execution from the kickoff
    # planning=True
)

## Query Time

In [17]:
result = blog_crew.kickoff_for_each(inputs=weaviate_features)

print(result)

<frozen importlib._bootstrap_external>:781: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x116d8a7a0>
<frozen importlib._bootstrap_external>:781: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x1333e5b70>
<frozen importlib._bootstrap_external>:781: ResourceWarning: unclosed database in <sqlite3.Connection object at 0x1333e5120>


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry researcher focused on biomedical trends and their applications in AI                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Conduct a thorough research about MUVERA                                                               │
│          Make sure you find any interesting and relevant information using the web and Weaviate blogs.          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Industry researcher focused on biomedical trends and their applications in AI                           │
│                                                                                                                 │
│  Thought: I need to gather information about MUVERA to understand its relevance and implications within the     │
│  biomedical industry. I will start by searching the internet for any existing information on MUVERA.            │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "MUVERA biomedical application features"                                                     │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'MUVERA biomedical application features', 'type': 'search', 'num': 10, 'engine':    │
│  'google'}, 'organic': [{'title': 'MUVERA: Making multi-vector retrieval as fast as single-vector search',      │
│  'link': 'https://research.google/blog/muvera-making-multi-vector-retrieval-as-fast-as-single-vector-search/',  │
│  'snippet': 'We introduce MUVERA, a state-of-the-art retrieval algorithm that reduces complex multi-vector      │
│  retrieval back to single-vector maximum inner product search.', 'position': 1}, {'title': 'Google MUVERA       │
│  (Multi-Vector Retrieval Architecture) - Guide', 'link': 'https://thatware.co/google-muvera-detailed-guide/',   │
│  'snippet': "MUVERA's vector-based breakdown ensures results go beyond generalized health advice. Instead, it   │
│  surfaces peer-reviewed research, expert medical commentary, and ...", 'position': 2}, {'title': "Google's      │
│  Muvera: Revolutionizing Search with Multimodal AI", 'link':                                                    │
│  'https://tbits.co.in/our-blogs/googles-muvera-revolutionizing-search-with-multimodal-ai', 'snippet': 'Muvera   │
│  is a multimodal AI model developed by Google that can process and generate multiple forms of media, including  │
│  text, images, and videos.', 'position': 3}, {'title': 'SmolLM3 from HuggingFace - by Bugra Akyildiz - MLOps    │
│  Newsletter', 'link': 'https://mlops.substack.com/p/smollm3-from-huggingface', 'snippet': 'MUVERA bridges the   │
│  gap by reducing multi-vector retrieval to single-vector MIPS through a mathematically principled approach.     │
│  The central ...', 'position': 4}, {'title': "Google's latest technology “MUVERA”! A new-age algorithm ... -    │
│  F2T", 'link': 'https://www.f2t.jp/en/blog-post/muvera', 'snippet': "So what Google developed is this “MUVERA   │
│  (Multi-Vector Retrieval Algorithm).” MUVERA's greatest invention is that it can process as fast as ...",       │
│  'position': 5}, {'title': "A Beginner's Guide to the MUVERA Algorithm in 2026 | Content Whale", 'link':        │
│  'https://content-whale.com/blog/muvera-algorithm-beginners-guide-2025/', 'snippet': 'It achieves 90% faster    │
│  p...                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insight analyst exploring innovations in finance, wealth tech, and regulatory tech                      │
│                                                                                                                 │
│  Thought: I have found relevant information regarding Hybrid Search, particularly from Weaviate and other       │
│  sources that highlight its applications in finance and wealth tech.                                            │
│                                                                                                                 │
│  Using Tool: WeaviateVectorSearchTool                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "query": "Hybrid Search applications in finance wealth tech"                                                 │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "author": null,                                                                                              │
│    "content": "---\ntitle: Hybrid Search Explained\nslug: hybrid-search-explained\nauthors: [erika]\ndate:      │
│  2023-01-03\ntags: ['concepts', 'search']\nimage: ./img/hero.png\ndescription: \"Learn about the new hybrid     │
│  search feature that enables you to combine dense and sparse vectors to deliver the best of both search         │
│  methods.\"\n---\n![Hybrid search](./img/hero.png)\n\n<!-- truncate -->\n\nHybrid search is a technique that    │
│  **combines** multiple search algorithms to improve the accuracy and relevance of search results. It uses the   │
│  best features of both keyword-based search algorithms with vector search techniques. By leveraging the         │
│  strengths of different algorithms, it provides a more effective search experience for users. The [Hybrid       │
│  search](/developers/weaviate/api/graphql/search-operators#hybrid) feature was introduced in Weaviate 1.17. It  │
│  uses sparse and dense vectors to represent the meaning and context of search queries and documents."           │
│  }{                                                                                                             │
│    "author": null,                                                                                              │
│    "content": ":::\n\n## How does hybrid search work, exactly? ![Hybrid search main image - a figurative image  │
│  of Weaviate bot combining results from vector and keyword search to produce hybrid                             │
│  search](./img/hybrid-search-illurated.png)\n\nHere is an example of a hybrid search:\n\n```python\nresponse =  │
│  (\n    client.query\n    .get(\"JeopardyQuestion\", [\"question\", \"answer\"])\n                              │
│  .with_hybrid(query=\"food\", alpha=0.5)\n    .with_limit(5)\n    .do()\n)\n```\n\nAs mentioned, a hybrid       │
│  search is really two searches under-the-hood. It performs a vector search (similar to `nearText` or            │
│  `nearVector` in Weaviate) to find most similar objects to the vector of your query. Meanwhile, it also         │
│  performs a keyword search, which ranks results based on how often the query terms occur. In other words, a     │
│  hybrid search performs both of these searches and combines the results."                                       │
│  }{                                                                                                             │
│    "author": null,                                                                                              │
│    "content": "We have prepared [this short                                                                     │
│  survey](https://docs.google.com/forms/d/e/1FAIpQLSfxekC2eh5Xkl0i...                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insight analyst exploring innovations in finance, wealth tech, and regulatory tech                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Based on the gathered information, here is a comprehensive analysis of how Hybrid Search can be particularly   │
│  beneficial in the finance, wealth tech, and regulatory tech sectors, specifically leveraging Weaviate's        │
│  capabilities.                                                                                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Industry-Specific Analysis: The Usefulness of Hybrid Search in Finance and Wealth Tech**                     │
│                                                                                                                 │
│  In the ever-evolving landscape of finance and wealth technology, the necessity for sophisticated data          │
│  retrieval methods has never been more critical. Hybrid search, a feature that merges traditional               │
│  keyword-based search with vector search, presents a transformative opportunity for financial                   │
│  applications—addressing challenges related to accuracy, relevance, and efficiency in data retrieval.           │
│                                                                                                                 │
│  ### Key Advantages of Hybrid Search in Finance and Wealth Tech:                                                │
│                                                                                                                 │
│  1. **Enhanced Search Accuracy**:                                                                               │
│     In financial services and wealth management, professionals need precise information to make critical        │
│  decisions. Hybrid search significantly improves search outcomes by combining the semantic understanding        │
│  capabilities of vector searches with keyword accuracy. For example, when searching for "risk management        │
│  strategies," hybrid search can yield both specific regulatory documents and broader academic studies,          │
│  ensuring that users receive comprehensive yet relevant results.                                                │
│                                                                                                                 │
│  2. **Contextual Understanding of Financial Terminology**:                                                      │
│     The financial sector is laden with specialized jargon and complex terminologies. Hybrid search excels in    │
│  interpreting contextual nuances, which is vital when different terms may have varying implications. For        │
│  instance, the term "derivatives" may relate to different financial instruments or risk management techniques.  │
│  The capability of hybrid search to discern these subtleties enhances the overall quality of search results     │
│  for practitioners navigating complex datasets.                                                                 │
│                                                                                                                 │
│  3. **Operational Efficiency in Research and Compliance

[CrewOutput(raw="I have collected detailed insights about MUVERA both from the internet and the Weaviate database. Now, I'll consolidate this information to produce a comprehensive analysis related to the financial sector.\n\n---\n\n### **Analysis of MUVERA and Its Applications in the Financial Industry**\n\nThe MUVERA (Multi-Vector Retrieval Algorithm) marks a significant advancement in information retrieval technologies, particularly relevant for industries like finance, where rapid access to large datasets is critical for effective decision-making. In financial markets, the ability to quickly interpret vast amounts of data—ranging from market trends, regulatory information, and customer preferences—can make a substantial difference in outcomes. MUVERA addresses several key challenges faced by the financial sector.\n\n### **Key Features and Benefits of MUVERA**\n\n1. **Enhanced Retrieval Speed and Efficiency**:\n   MUVERA transforms complex multi-vector embeddings into simpler single